In [1]:
import picamera
import picamera.array

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
import time
import numpy as np
import cv2
from PIL import Image
import keras
from keras import backend as K
from keras.layers.core import Dense
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Model
from keras.applications import imagenet_utils
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Input
import cv2
import numpy as np
import os


Using TensorFlow backend.
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.4 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: builtins.type size changed, may indicate binary incompatibility. Expected 432, got 412
  return f(*args, **kwds)


In [4]:
p = 'dataset'
TRAINING_DATA = []  # Example array to be trained
TRAINING_LABELS = []  # Label array
NUM_CLASSES = 0
count = [0, 0]

NUM_CLASSES = 2
LEARNING_RATE = 0.0001
EPOCHS = 10
DENSE_UNITS = 100


In [5]:
def get_image(path):
    print("get image called")
    global NUM_CLASSES
    image_path = [os.path.join(path, f) for f in os.listdir(path)]
    classes = [p.split('/')[1] for p in image_path]
    NUM_CLASSES = len(classes)

    print(NUM_CLASSES)
    print(classes)
    for cls in classes:
        pp = path + "/" + cls
        print("class={}".format(pp))
        image_path = [ os.path.join(pp, f) for f in os.listdir(pp) if f.endswith('.jpg') ]
#         print(image_path)
        print(len(image_path))
        for img in image_path:
#             print("path={}".format(img))
            add_example(prepare_frame(cv2.imread(img)), int(cls))

In [6]:
def add_example(example, label):  # add examples to training data set
    encoded_y = keras.utils.np_utils.to_categorical(label, num_classes=NUM_CLASSES)  # make one-hot
    TRAINING_LABELS.append(encoded_y)
    TRAINING_DATA.append(example[0])
#     print('> add example for label %d' % label)
    count[label] += 1


def prepare_frame(frame):
    img = Image.fromarray(frame, 'RGB')
    img = img.resize((224, 224))
    img_array = np.array(img)
    img_array_extended = np.expand_dims(img_array, axis=0).astype('float32')
    processed = keras.applications.mobilenet.preprocess_input(img_array_extended)
    return processed

In [7]:
get_image("dataset")

get image called
2
['1', '0']
class=dataset/1
23
class=dataset/0
11


In [8]:
from keras import backend as K
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Dense,GlobalAveragePooling2D,Dropout,SeparableConv2D,BatchNormalization, Activation, Dense
from keras.applications.mobilenet import MobileNet
from keras.optimizers import Adam

def load_model():
    mobilenet = keras.applications.mobilenet.MobileNet()
#     base_model = mobilenet
#     x=base_model.output
#     # Add some new Fully connected layers to 
#     x=GlobalAveragePooling2D()(x)
#     x=Dense(1024,activation='relu')(x)
#     x = Dropout(0.25)(x)
#     x=Dense(512,activation='relu')(x) 
#     x = Dropout(0.25)(x)
#     preds=Dense(2, activation='softmax')(x) #final layer with softmax activation
#     model=Model(inputs=base_model.input,outputs=preds)    
    flatten = Flatten(input_shape=(7, 7, 1024))(mobilenet.get_layer('conv_pw_13_relu').output)
    fc1 = Dense(DENSE_UNITS, activation='relu')(flatten)
    fc2 = Dense(NUM_CLASSES)(flatten)
    output = Activation('softmax')(fc2)
    model = Model(mobilenet.input, output)
#     # make all layers untrainable by freezing weights (except for last two layers)
    for l, layer in enumerate(model.layers[:-3]):
        layer.trainable = False
    # ensure the last layer is trainable/not frozen
    for l, layer in enumerate(model.layers[-3:]):
        layer.trainable = True
    return model

In [10]:
epochs = 10
learning_rate = 0.0005
decay_rate = learning_rate / epochs

model = load_model()
for i,layer in enumerate(model.layers):
    print("{}: {}".format(i,layer))
# model.summary()

# for layer in model.layers[:87]:
#     layer.trainable=False
# for layer in model.layers[87:]:
#     layer.trainable=True
opt = Adam(lr=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=None, decay=decay_rate, amsgrad=False)    
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


0: <keras.engine.input_layer.InputLayer object at 0x54f2efb0>
1: <keras.layers.convolutional.ZeroPadding2D object at 0x5603a810>
2: <keras.layers.convolutional.Conv2D object at 0x5609fb50>
3: <keras.layers.normalization.BatchNormalization object at 0x5609fad0>
4: <keras.layers.advanced_activations.ReLU object at 0x5609f6f0>
5: <keras.layers.convolutional.DepthwiseConv2D object at 0x56055ab0>
6: <keras.layers.normalization.BatchNormalization object at 0x56070ef0>
7: <keras.layers.advanced_activations.ReLU object at 0x56070370>
8: <keras.layers.convolutional.Conv2D object at 0x56020bf0>
9: <keras.layers.normalization.BatchNormalization object at 0x55fcc9d0>
10: <keras.layers.advanced_activations.ReLU object at 0x55fcccf0>
11: <keras.layers.convolutional.ZeroPadding2D object at 0x55f7ff10>
12: <keras.layers.convolutional.DepthwiseConv2D object at 0x55f7fab0>
13: <keras.layers.normalization.BatchNormalization object at 0x55f602b0>
14: <keras.layers.advanced_activations.ReLU object at 0x55f

In [11]:
model.fit(np.array(TRAINING_DATA), np.array(TRAINING_LABELS), epochs=epochs, batch_size=8)


Epoch 1/10
34/34 [==============================] - 26s 755ms/step - loss: 6.5434 - accuracy: 0.6471
Epoch 2/10
34/34 [==============================] - 22s 653ms/step - loss: 7.9717e-05 - accuracy: 1.0000
Epoch 3/10
34/34 [==============================] - 22s 660ms/step - loss: 0.3928 - accuracy: 0.9118
Epoch 4/10
34/34 [==============================] - 22s 653ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 5/10
34/34 [==============================] - 22s 651ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 6/10
34/34 [==============================] - 22s 652ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 7/10
34/34 [==============================] - 22s 650ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 8/10
34/34 [==============================] - 22s 649ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 9/10
34/34 [==============================] - 22s 660ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 10/10
34/34 [==============================] - 22s 652ms

In [12]:
model.predict

<bound method Model.predict of <keras.engine.training.Model object at 0x5424f030>>

In [13]:
model.save('man-sleep.h5')

In [12]:

# # with picamera.PiCamera() as camera:
# #     camera.resolution = (1024, 768)
# #     camera.framerate = 24    
# #     with picamera.array.PiRGBArray(camera) as output:      
# #         camera.capture(output, 'rgb')
# #         print('Captured %dx%d image' % (output.array.shape[1], output.array.shape[0]))
# #         frame = output.array
# #         jpeg_image = bgr8_to_jpeg(frame)
# #         image_widget.value = jpeg_image
        
# with picamera.PiCamera() as camera:
#     camera.resolution = (480, 320)
#     camera.framerate = 24        
#     with picamera.array.PiRGBArray(camera) as output:
#         while True:        
#             camera.capture(output, 'rgb')
#             print('Captured %dx%d image' % (output.array.shape[1], output.array.shape[0]))
#             frame = output.array
#             processed_image = prepare_frame(frame)  # prepare frame
#             prediction = model.predict(processed_image)
#             result = np.argmax(prediction)  # imagenet_utils.decode_predictions(prediction)
#             print('predict class= %d' % result)
#             output.truncate(0)


In [13]:
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
import os
text = widgets.Text(
    value='Hello World',
    placeholder='Type something',
    description='String:',
    disabled=False
)


In [8]:
from keras.models import load_model

# Creates a HDF5 file 'my_model.h5'
# model.save('my_model.h5')

# # Deletes the existing model
# del model  

# Returns a compiled model identical to the previous one
model = load_model('man-sleep.h5')

Using TensorFlow backend.
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

In [11]:
import paho.mqtt.client as mqtt
mqttc = mqtt.Client()
mqttc.connect("mqtt.cmmc.io", 1883, 60)

mqttc.publish("CMMC/PLUG-001/$/command", "ON")

In [14]:
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
import os

# from __future__ import print_function
from imutils.video import VideoStream
from imutils.video import FPS
from picamera.array import PiRGBArray
from picamera import PiCamera
import argparse
import imutils
import time
import cv2
from imutils.video import FPS
import datetime
import time
import numpy as np
import cv2
from PIL import Image

#image = widgets.Image(format='jpeg', width=224, height=224)  # this width and height doesn't necessarily have to match the camera
#camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])

# initialize the camera and stream
camera = PiCamera()
# camera.resolution = (320, 240)
camera.framerate = 32
camera.start_preview()
# rawCapture = PiRGBArray(camera, size=(320, 240))
rawCapture = PiRGBArray(camera)
stream = camera.capture_continuous(rawCapture, format="bgr",
use_video_port=True)

image_widget = widgets.Image()
display(text)
display(image_widget)


for (i, f) in enumerate(stream):
    frame = f.array
    image_widget.value =  bgr8_to_jpeg(frame)
    processed_image = prepare_frame(frame)  # prepare frame
    t1 = datetime.datetime.now()
    prediction = model.predict(processed_image)
    t2 = datetime.datetime.now()
    result = np.argmax(prediction)  # imagenet_utils.decode_predictions(prediction)
    text.value = "class=" + str(result)
    if str(result) == "0":
        mqttc.publish("CMMC/PLUG-001/$/command", "ON")
    else:
        mqttc.publish("CMMC/PLUG-001/$/command", "OFF")
    rawCapture.truncate(0)

PiCameraMMALError: Failed to enable connection: Out of resources